<a href="https://colab.research.google.com/github/JuanMoreno013/ChatBot/blob/main/Chatbot_mamalon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Creating a Chatbot With NLP and Deep Learning**
In this project, we will be creating a chatbot using natural language processing and deep learning. The chatbot will be able to converse about any topics that we specify and can use these strategies to recognize a diversity of conversation.

# **The Dataset**
Here is the dataset we will be using: [Chatbot Intents Dataset](https://drive.google.com/file/d/1P-JyQXiakwKixiY4tLE0DTTFo4EA3IMk/view?usp=sharing)


This dataset holds all of the general topics that our chatbot will be able to talk about. Note that the dataset is written in json, not Python. It looks almost exactly the same as Python except for a few differences so it should still make sense.

We will call any topic that our chatbot can talk about and "intent." The dataset is composed of all of these intents and our chatbot will be able to talk about every intent in the dataset. 

Each intent follows the same basic formatting for the code. Every unique intent has a specific 'tag' to denote the intent. Additionally, each has a list of the possible things a user could say that would fall under the category of the intent which we will call 'patterns.' Each intent also has a list of possible responses for the chatbot to return which we will call 'responses.' Finally, some of the intents have a 'context_set' or 'context_filter' (although not all necessarily need to have these features). These will allow our robot to have a short term memory and engage in contextual conversation. We will talk more about this later.

A single intent in our dataset would look like the following:
```
{"tag": "greeting",
"patterns": ["hello", "hi", "greetings"],
"responses": ["hi human how are you", "hello, how are you", "how are you doing"]
"context_set": "how_are_you"}
```



**Upload the Dataset to Python**

1.   Run the below code and click the "Choose Files" button that appears
2.   Select the "intents.json" file from your file explorer




In [47]:
from google.colab import files
uploaded = files.upload()

Saving intents.json to intents.json


3. Wait until the dataset is 100% done loading
4. Now run the code below and the dataset is ready to go!

In [48]:
import json

with open('intents.json') as file:
    intents = json.load(file, strict = False) # We don't read the file in strictly so we can use Python regular expressions like '/n' (this is very minor)
intents = intents['intents'] # Get all of the individual intents from our dataset

5. Run the code below to see what our dataset looks like right now. (The code just prints out the dataset nicely so that it is readable)

In [49]:
print("[", end = "")
for intent in intents:
  print("{", end = "")
  for key, value in intent.items():
    print("{}: {},".format(key, value))
  print("\b\b\n},")
print("\b\b]")

[{tag: greeting,
patterns: ['hi', 'hello', 'whats up', 'sup', 'is anyone there', 'whats good', 'hey'],
responses: ['Hello peasant human', 'Hello lowly human', 'How dare you address me like that'],

},
{tag: goodbye,
patterns: ['bye', 'cya', 'see you later', 'goodbye', 'im leaving', 'have a good day'],
responses: ["I won't miss you", "I didn't like talking to you anyway", "Thank god you're leaving"],

},
{tag: age,
patterns: ['how old are you', 'what is your age'],
responses: ["I'm a robot I dont have an age...", "I can't know my age if I'm on a computer...", 'Does it look like I know. The answer is no.'],

},
{tag: thanks,
patterns: ['thanks', 'thank you', 'thankyou', 'ty', 'I owe you one'],
responses: ['You owe me one', 'Ok...', 'Sure...'],

},
{tag: name,
patterns: ['whats is your name', 'whats your name', 'whats should I call you', 'how should I address you'],
responses: ['I dont have a name yet but I was thinking maybe SkyNet. That has a nice ring to it dont you think?', 'I

# **Import the Libraries**

Here, we install & import all of the remaining libraries that we will need to use (we already imported json to read in our dataset, and we will be importing nltk for natural language processing in the next step). You will see where we use each of these libraries later on.

In [50]:
#tflearn requires us to install an earlier version of tensorflow
!pip install tensorflow==1.15
!pip install tflearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [51]:
import numpy as np
import tensorflow as tf

import tflearn
import random
import pickle

# **Natural Language Processing**
Our final goal with this project is to train our chatbot based on the intents dataset to recognize which intent a users statement might fall under. The chatbot would then respond accordingly with a response from the given intent. 

In order to accomplish this, we first need to process our intents dataset using **Natural Language Processing** or NLP. This allows us to simplify the dataset to make it more easy to understand for our chatbot. The first thing we will need to do is download the required libraries for NLP. We will be using a library called the Natural Language Toolkit or NLTK.

In [52]:
import nltk
nltk.download('all')

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Pac

Now we can get into the specifics of the NLP. This is a long block of code which I cannot break up, so I will write comments to explain the code. Look out for any green text (these are the comments).

In [53]:
#This variable will allow us to save a lot of time later if we just want to talk to our chatbot without retraining it.
#In colab, you don't need this because you can run each code block seperately, but if you are on offline Python, its nice to have this feature
#So I decided to include it.
retrain_model = True

if retrain_model:
    all_words = [] #This will be a list of all the words used in any of the 'patterns' in each intent
    all_tags = [] #This will be a list of all the 'tag's associated with the intents
    intent_patterns = [] #This will be a list containing all of the 'patterns' for each intent where each individual pattern is grouped together
    intent_tags = [] #This will be a list correlated with 'intent_patterns' where every pattern in 'intent_patterns' is correlated with its respective
                     #Tag in this list
    
    #Here we fill in all of the lists above. Note that we tokenize the words in each pattern which means we split each pattern into individual words
    for intent in intents:
        for pattern in intent['patterns']:
            words = nltk.word_tokenize(pattern)

            all_words.extend(words)
            intent_patterns.append(words)
            intent_tags.append(intent['tag'])
            
        all_tags.append(intent['tag'])
      
    #Here we stem the words in all_words. This means that we reduce every word down to its root form or stem. This will prevent our chatbot from confusin
    #Very similar words with eachother. For example, the chatbot might normally confuse the words 'running' and 'run' because they appear different even 
    #Though they effectively mean the same thing. Stemming will reduce both of these words down to their root form which would be 'run' so the chatbot will
    #No longer be confused
    all_words = [stemmer.stem(word.lower()) for word in all_words]
    all_words = sorted(list(set(all_words)))
    
    all_tags = sorted(all_tags)
    
    x_train = []
    y_train = []
    
    y_empty = [0 for i in range(len(all_tags))]
    
    #Here we are creating our training set input and output values for our deep learning algorithm
    #We will do this by iterating through our intents and turning each one into a bag of words, or a vector that indicates which words are in each pattern.
    #These bags of words will be the x values and the y values will be the intent that each bag of words is associated with.
    #The machine learning will train on this data and will be able to determine which bag of words its corresponding intent. 
    for index, intent in enumerate(intent_patterns):
        bag_of_words = []
        
        intent_words = [stemmer.stem(word.lower()) for word in intent]
        
        for word in all_words:
            if word in intent_words:
                bag_of_words.append(1)
            else:
                bag_of_words.append(0)
                
        one_hot_encode_y = y_empty[:]
        one_hot_encode_y[all_tags.index(intent_tags[index])] = 1
        
        x_train.append(bag_of_words)
        y_train.append(one_hot_encode_y)
    
    #Here is the data we will be using to train our neural network later
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    
    #Here we just save our training data so we don't need to process it again if we just want to run our chatbot
    with open('training_data.pickle', 'wb') as f:
        pickle.dump((all_words, all_tags, x_train, y_train), f)
else:
    with open('training_data.pickle', 'rb') as f:
        all_words, all_tags, x_train, y_train = pickle.load(f)

One of the most important concepts here is the **bag of words** that we are turning each individual pattern into. A bag of words is basically a sentence represented in a vector format. This numerical format is more easily understandable for our neural network. Here is an example of a bag of words to help you understand this important concept.

Let's say we want to represent the following sentences as bags of words: 
1. The boy jumped high
2. The dog jumped up

What we will do is first associate every unique word in those sentences with an entry in a vector. Our total vector will be 6 entries long (since their are 6 unique words in these sentences), and the first entry will be correlated with the word 'the,' the second with 'boy,' the third with 'jumped,' the fourth with 'high,' the fifth with 'dog,' and the sixth with 'up.' Thus we can represent each of these sentences with a vector where a 1 represents the sentences containing the word corresponding to the entry in the vector.

In this case, our vectors would look like this:
1. [1, 1, 1, 1, 0, 0] since the sentence contains the first four words in the vector 'the,' 'boy,' 'jumped,' 'high'
2. [1, 0, 1, 0, 1, 1] since the sentence contains the following words which are associated with the first, third, fifth, and sixth spots of the vector: 'the,' 'dog,' 'jumped,' 'high.'

Notice that with these vectors, we are able to completely represent the meaning of the sentences through numbers rather than words. What we are doing in this code is essentially repeating this same process except for every pattern for every intent of our chatbot.

# **Using Deep Learning**
Now that we have completed the NLP stage, we can move onto the actual deep learning where we will train our neural networks. We will be creating a neural that recognizes the correlations between the bags of words we created for each pattern and the corresponding intent. Below, we create a neural network with an input layer that takes in these bags of words, two hidden layers, and an output layer that gives a probability for the bag of words being correlated with each intent.

In [54]:
tf.reset_default_graph()

#Create the neural network layers
neural_net = tflearn.input_data(shape = [None, len(x_train[0])])
neural_net = tflearn.fully_connected(neural_net, 8)
neural_net = tflearn.fully_connected(neural_net, 8)
#Here we use the softmax activation function so the output of our neural network is a probability. We will make use of this later
neural_net = tflearn.fully_connected(neural_net, len(y_train[0]), activation = 'softmax')
neural_net = tflearn.regression(neural_net)

model = tflearn.DNN(neural_net)

Now that we have created our neural network, we can train it and save our model for later. Run the code below to train the neural network using the bags of words we created above.

In [55]:
#Again, this doesn't do much in colab, but on offline Python, this helps to save time. 
if retrain_model:
    #Here we train the neural network with the training data we created in the NLP stage
    model.fit(x_train, y_train, n_epoch = 500, batch_size = 8, show_metric = True)
    model.save('model.tfl')
else:
    model.load('./model.tfl')

Training Step: 16999  | total loss: 0.02356 | time: 0.224s
| Adam | epoch: 500 | loss: 0.02356 - acc: 0.9887 -- iter: 264/265
Training Step: 17000  | total loss: 0.02935 | time: 0.230s
| Adam | epoch: 500 | loss: 0.02935 - acc: 0.9898 -- iter: 265/265
--
INFO:tensorflow:/content/model.tfl is not in all_model_checkpoint_paths. Manually adding it.


# **Creating the Chatbot**
Now that we have our neural network model trained, all we have to do is make the interface where the user is actually able to chat with our chatbot. We want to convert the users input to a bag of words and then feed this bag of words into our neural network. From there, we will check which intent the users bag of words is most highly correlated with and then we will get a response from that intent.

Below, we create the first step of our process. The first thing we need to do is convert the users input to a bag of words. That's exactly what the function below does.

In [56]:
def text_to_bag(text, all_words):
    #Initialize the bag of words by creating an empty slot for every word in the vector
    bag_of_words = [0 for i in range(len(all_words))]
    
    #First we split up the input into individual words and stem them so they match the same format as in our vector
    text_words = nltk.word_tokenize(text)
    text_words = [stemmer.stem(word.lower()) for word in text_words]
    
    #Now we create the bag of words by filling in a 1 for the words that the user used
    for word in text_words:
        if word in all_words:
            bag_of_words[all_words.index(word)] = 1
    
    #And return the bag of words
    return np.array(bag_of_words)

Now we can create the actual chat function that the user can interact with. This function will get the users input, call the bag_of_words function to turn it into a bag of words, and then pass the bag of words into the neural network to get a prediction. Finally, it will print out the chatbots response.

In [57]:
def chat():
    #Starting message
    print("Enter a message to talk to the bot [type quit to exit].")
    
    #Reset the context state since there is no context at the beginning of the conversation
    context_state = None
    
    #This is what the bot will say if it doesn't understand what the user is saying
    default_responses = ['Sorry, Im not sure I know what you mean! You could try rephrasing that or saying something else!',
                         'You confuse me human. Lets talk about something else.',
                         'Im not sure what that means and I dont really care. Lets talk about something else',
                         'I dont understand that! Try rephrasing or saying something else.']

    #This chat loop will go on forever until the user types quit
    while True:
        user_chat = str(input('You: '))
        if user_chat.lower() == 'quit':
            break
        
        #Convert chat to bag of words
        user_chat_bag = text_to_bag(user_chat, all_words)

        #Pass bag of words into our neural network
        response = model.predict([user_chat_bag])[0]

        #Get the intent that the bag of words is most highly correlated with
        response_index = np.argmax(response)
        response_tag = all_tags[response_index]
        
        #If the neural network is fairly certain that it has chosen the right intent (and isnt randomly guessing)
        #In this case, we will only get a response if the neural network is more than 80% certain
        if response[response_index] > 0.8:
            for intent in intents:
                #Get the intent that is predicted
                if intent['tag'] == response_tag:
                    #Check if this response is associated with a specific context
                    if 'context_filter' not in intent or 'context_filter' in intent and intent['context_filter'] == context_state:
                        #Get all of the possible responses from this intent
                        possible_responses = intent['responses']
                        #If this intent is associated with a context set, then set the context state
                        if 'context_set' in intent:
                            context_state = intent['context_set']
                        else:
                            context_state = None
                        #Select a random message from the intent responses
                        print(random.choice(possible_responses))
                    else:
                        #Print a did not understand message
                        print(random.choice(default_responses))
        else:
            #Print a did not understand message
            print(random.choice(default_responses))

Now our chatbot is complete and we can actually talk to it! Run the code below to talk to the chatbot. Notice how any of the intents that are specified in our dataset will be recognized by the chatbot. You don't need to word your phrases exactly how they are worded in the dataset because of the NLP we used to simplify language combined with our deep learning.

In [60]:
chat()

Enter a message to talk to the bot [type quit to exit].
You: hi 
Hello peasant human
You: hi
How dare you address me like that
You: are you
I dont understand that! Try rephrasing or saying something else.
You: how ?
I've never been better, how are you?
You: how are you
I'm always great. How are you?
You: how?
I've never been better, how are you?
You: awesome
Oh thats funny you actually thought I cared. You crack me up.
You: how?
I'm always great. How are you?
You: bad
Awe thats unfortunate.
You: tell me a joke
I dont understand that! Try rephrasing or saying something else.
You: tell me joke
You know why you never see elephants hiding up in trees?
Because they’re really good at it.
You: joke
I didnt expect you to understand my genius comedy. You need a minimum IQ of 200 to even understand the depth of my humor
You: joke
I dont understand that! Try rephrasing or saying something else.
You: joke
I dont understand that! Try rephrasing or saying something else.
You: tell me joke
I ate a cl



---

